# Playbook Explorer

This notebook is intended to be a live example of how to work with SysML v2 models at analysis-time. For these purposes, the following terms are introduced:
* An *interpretation* is the mapping of user model symbols (the "M1 model" in OMG-speak) into semantically-correct symbols that represent real world objects meant to conform to the model (the "M0" in OMG-speak). Interpretation semantics are inspired by https://www.w3.org/TR/owl2-direct-semantics/ and are mostly similar.
* A *sequence* for an interpretation contains *atoms* or *instances* that match to real world things. Reading a sequence from left to right provides a set of nested contexts for the atoms that is important to the interpretation. For example [Rocket#0, LS#3] is a 2-sequence to describe facts around the LS#3 atom when it is considered in context for Rocket#0. This is an important idea for the SysML time and occurrence model where one may want to see how values change under different conditions.

This is a notebook that walks through the random interpretation generator to help developers working on their own interpreters.

## Example Model

The model that is used for this example is a very simple parts model.

    package 'Simple Parts Model' {
        package 'Fake Library' {
            part def Part;
            port def Port;
            connection def Connection;
        }
    
        part 'Power Group' : 'Fake Library'::Part {
            part 'Power Source' : 'Fake Library'::Part {
                port 'Power Out' : 'Fake Library'::Port;
            }
            part 'Power User' : 'Fake Library'::Part {
                port 'Power In' : 'Fake Library'::Port;
            }

            connection powerToUser : 'Fake Library'::Connection connect 'Power Source'::'Power Out' to 'Power User'::'Power In';
        }
    
    }

## Imports

Import key modules, functions, and classes from the PyMBE library:

In [ ]:
from pathlib import Path
import networkx as nx

import pymbe.api as pm

from pymbe.client import SysML2Client
from pymbe.graph.lpg import SysML2LabeledPropertyGraph
from pymbe.interpretation.interp_playbooks import (
    build_expression_sequence_templates,
    build_sequence_templates,
    random_generator_playbook,
)
from pymbe.interpretation.results import *
from pymbe.label import get_label_for_id
from pymbe.query.metamodel_navigator import feature_multiplicity
from pymbe.query.query import (
    roll_up_multiplicity,
    roll_up_upper_multiplicity,
    roll_up_multiplicity_for_type,
    get_types_for_feature,
    get_features_typed_by_type,
)
from pymbe.local.stablization import build_stable_id_lookups

## Client Setup

The example here uses a local copy of the JSON file obtained by a GET operation on the SysML v2 API at:
http://sysml2-sst.intercax.com:9000/projects/a4f6a618-e4eb-4ac8-84b8-d6bcd3badcec/commits/c48aea9b-42fb-49b3-9a3e-9c39385408d7/elements?page[size]=5000

Create the client and load local data.

In [ ]:
helper_client = SysML2Client()

simple_parts_file = Path("..") / "tests/fixtures/Simple Parts Model.json"

helper_client._load_from_file(simple_parts_file)

Create a graph representation of the model and load it into memory.

In [ ]:
lpg = SysML2LabeledPropertyGraph()
lpg.model = helper_client.model

This is just a helper to make abbreviations more legible.

In [ ]:
shorten_pre_bake = {}

Create an interpretation of the Kerbal model using the random generator playbook. In general, this randomly selects:
- The ratios of partitioning abstract classifier sequence sets into concrete sets. For example, one draw may choose 2 liquid stages and 3 solids.
- The number of sequences to create for a given feature multiplicity. For example, draw 2 for a 0..8 engines : Liquid Engine PartUsage.

The playbook also attempts to make sequences created obey the Subsetting relationship (elements marked with subsets in M1 model should have their interpretation sequences entirely included within the interpretation sequences of the superset).

In [ ]:
SIMPLE_MODEL = "Model::Simple Parts Model::"
PARTS_LIBRARY = "Model::Simple Parts Model::Fake Library::"

name_to_id_lookup = build_stable_id_lookups(lpg)[1]
name_to_id_lookup

connection_id = name_to_id_lookup[f"{PARTS_LIBRARY}Connection <<ConnectionDefinition>>"]
power_source_id = name_to_id_lookup[f"{SIMPLE_MODEL}Power Group: Part::Power Source: Part <<PartUsage>>"]
power_user_id = name_to_id_lookup[f"{SIMPLE_MODEL}Power Group: Part::Power User: Part <<PartUsage>>"]
part_id = name_to_id_lookup[f"{PARTS_LIBRARY}Part <<PartDefinition>>"]
power_in_id = name_to_id_lookup[f"{SIMPLE_MODEL}Power Group: Part::Power User: Part::Power In: Port <<PortUsage>>"]
connect_use_id = name_to_id_lookup[f"{SIMPLE_MODEL}Power Group: Part::powerToUser: Connection <<ConnectionUsage>>"]
power_group_id = name_to_id_lookup[f"{SIMPLE_MODEL}Power Group: Part::powerToUser: Connection <<ConnectionUsage>>"]

In [ ]:
from pymbe.query.query import get_types_for_feature, get_features_typed_by_type, roll_up_multiplicity_for_type

In [ ]:
m0_interpretation = random_generator_playbook(
    lpg,
    shorten_pre_bake,
)

To see how sequences are structured, the cell below renders sequences that show what type of atoms will fill particular positions in the sequence, as well as the maximum multiplicity (number of) sequences.

## Calculation Results Shown

The following cells are a series of displays of relevant features in the interpretation.

Show all interpretation sequence sets (limited to length of 5).

In [ ]:
for print_line in pprint_interpretation(m0_interpretation, lpg.model):
    print(print_line)

# Lets make that IBD

In [ ]:
from pprint import pprint
from collections import defaultdict


connector_usagetypes = ("ConnectionUsage", "SuccessionUsage")
elements = lpg.model.elements

connectors, m1_connectors = [], []
for id_, sequences in m0_interpretation.items():
    element = elements[id_]
    if element["@type"] in connector_usagetypes:
        connectors += [connector_usage for *_, connector_usage in sequences]
        src, tgt = element["connectorEnd"][:2]
        src, tgt = src["@id"], tgt["@id"]
        m1_connectors += [(element["@id"], src, tgt)]

to_parent = {}
for id_, sequences in m0_interpretation.items():
    element = elements[id_]
    if element["@type"] in ("PartUsage", "PortUsage"):
        for sequence in sequences:
            parent = sequence[0]
            for child in sequence[1:]:
                to_parent[child] = parent
                parent = child
    
pprint(dict(to_parent))

# Find M0 Source and Targets for Each M0 Connector
> This is hacky AF but it works...

In [ ]:
m0_connectors = []
for conn_usage, src, tgt in m1_connectors:
    for m0_conn_usage in m0_interpretation[conn_usage._id]:
        num_items = len(m0_conn_usage)
        for m0_src in m0_interpretation[src._id]:
            if m0_src[:num_items] == m0_conn_usage:
                m0_src = m0_src[-1]
                break
        for m0_tgt in m0_interpretation[tgt._id]:
            if m0_tgt[:num_items] == m0_conn_usage:
                m0_tgt = m0_tgt[-1]
                break
        m0_connectors += [(tuple(m0_conn_usage), m0_src, m0_tgt)]
m0_connectors

# Draw the Diagram

In [ ]:
import ipywidgets as ipyw
from pymbe.widget.diagram.parts import Part
from pymbe.widget.diagram.part_diagram import PartDiagram
from pymbe.widget.diagram.relationships import Relationship

import ipyelk as IE

from ipyelk.elements import Compartment, Edge, Label, Node, Port, NodeShape, NodeProperties
from ipyelk.elements.shapes import Widget

In [ ]:
m0_connectors

In [ ]:
diagram = IE.Diagram(layout={"height": "100%"})
diagram

In [ ]:
parts = {
    instance: Node(
        layoutOptions={
            "org.eclipse.elk.portLabels.placement": "INSIDE",
            "org.eclipse.elk.nodeSize.constraints": "NODE_LABELS PORTS PORT_LABELS MINIMUM_SIZE",
            "org.eclipse.elk.nodeLabels.placement": "H_CENTER V_CENTER",
        },
        labels=[
            Label(text=instance.name)
            # for part in (instance.name.split("#")[0], instance.name)
        ],
        # Leaving the commented out code for future reference
#         properties=NodeProperties(shape=Widget(
#             widget=ipyw.Text(),
#             width=200,
#             height=100,
#         )),
    )
    for instance in set([*to_parent, *to_parent.values()])
}

In [ ]:
part_diagram = PartDiagram()

compartments = {}
ports = {}


for instance, part in parts.items():
    parent = parts.get(to_parent.get(instance, None), None)
    if parent is None:
        part_diagram.add_child(part)
        continue
    if instance.name.startswith("Port"):
        ports[instance] = parent.add_port(
            Port(
                labels=[Label(
                    text=instance.name, #.split("#")[-1],
                )],
                layoutOptions={
                    "org.eclipse.elk.port.borderOffset": "-15"
                },
                height=15,
                width=15,
            )
        )
    else:
        if not hasattr(parent, "parts"):
            parent.parts = Node()
        parent.parts.add_child(part)

In [ ]:
self_edges = []
for connector, src, tgt in m0_connectors:
    edge = parent.add_edge(source=ports[src], target=ports[tgt])
    if ports[src].get_parent() == ports[tgt].get_parent():
        self_edges.append(edge)

In [ ]:
loader = IE.ElementLoader()
diagram.source = loader.load(part_diagram)
diagram.style = part_diagram.style
diagram.view.symbols = part_diagram.symbols

In [ ]:
[edge.id for edge in self_edges]

In [ ]:
val = diagram.pipe.pipes[0]
val.errors

--------------

> All below is for troubleshooting

In [ ]:
loader.default_root_opts

In [ ]:
loader.default_node_opts

In [ ]:
from ipyelk.elements import Edge, Label, Node, Port
from ipyelk import from_element


node = Node(
    labels=[Label(text="Node")],
    layoutOptions={
        'org.eclipse.elk.nodeSize.constraints': 'NODE_LABELS PORTS PORT_LABELS MINIMUM_SIZE',
        'org.eclipse.elk.hierarchyHandling': 'INCLUDE_CHILDREN',
    },
    height=200,
    width=200,
)
port1 = Port(
    labels=[Label(text="1")],
    width=10,
    height=10,
)

port2 = Port(
    labels=[Label(text="2")],
    width=10,
    height=10,
)
node.add_port(port1)
node.add_port(port2)

node.add_edge(source=port1, target=port2)

diagram = from_element(node)
diagram

In [ ]:
JSON(diagram.view.source.value.dict())

In [ ]:
diagram.pipe.exception